In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

In [13]:
import dataset_utils
import importlib

In [14]:
importlib.reload(dataset_utils)

<module 'dataset_utils' from 'd:\\University\\P4P\\P4P-mmWave\\MMWave_Radar_Human_Tracking_and_Fall_detection\\dataset_utils.py'>

In [15]:
DATA_DIR = "pointnet_data"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: sitting
processing class: standing


In [16]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [17]:
print(train_points)

[[[-2.4365e-01  1.4092e+00  5.8984e-01 -1.2524e-01  1.5100e+02]
  [-1.5625e-01  1.8936e+00  6.0156e-01 -1.2524e-01  1.6100e+02]
  [-1.6162e-01  1.4668e+00  6.1133e-01 -2.5049e-01  2.5200e+02]
  ...
  [-2.4365e-01  1.4199e+00  6.0156e-01 -1.2524e-01  1.2700e+02]
  [-3.2544e-01  1.4443e+00  1.9727e-01  0.0000e+00  1.4600e+02]
  [-1.6162e-01  1.4668e+00  6.1133e-01 -2.5049e-01  2.5200e+02]]

 [[-1.5625e-01  1.8936e+00  6.0156e-01 -1.2524e-01  1.6100e+02]
  [-1.3721e-01  1.6309e+00  6.5332e-01  0.0000e+00  2.1400e+02]
  [-2.2168e-01  1.5264e+00  5.8203e-01  0.0000e+00  1.4800e+02]
  ...
  [-2.1094e-01  1.6367e+00  6.3281e-01  0.0000e+00  2.4400e+02]
  [-2.2168e-01  1.5264e+00  5.8203e-01  0.0000e+00  1.4800e+02]
  [-2.4365e-01  1.3262e+00  5.2344e-01  0.0000e+00  1.3300e+02]]

 [[-4.5898e-01  1.4893e+00  6.6309e-01  1.2524e-01  2.2900e+02]
  [-8.7988e-01  2.3496e+00  6.6602e-01  2.5049e-01  2.3900e+02]
  [-4.0454e-01  1.6689e+00  3.6133e-01  0.0000e+00  1.4000e+02]
  ...
  [-4.1528e-01  1.

In [18]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [19]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [20]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [21]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [22]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# # 2. LSTM layer
# model.add(layers.LSTM(50, return_sequences=True))
# model.add(layers.LSTM(50))

# 3. Dense layers
model.add(layers.Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(1, activation='linear'))  # Output layer (you can adjust the output size and activation)


c:\Users\Yang Qian\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, None, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 32)       │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, None, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 1)        │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,785 (120.25 KB)

 Trainable params: 30,337 (118.50 KB)

 Non-trainable params: 448 (1.75 KB)

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, batch_size=16, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 3.6844 - mae: 1.4858 - val_loss: 0.4594 - val_mae: 0.5832
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9348 - mae: 1.0784 - val_loss: 0.3104 - val_mae: 0.5136
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4202 - mae: 0.9189 - val_loss: 0.2947 - val_mae: 0.5065
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0863 - mae: 0.8089 - val_loss: 0.2775 - val_mae: 0.4960
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8686 - mae: 0.7283 - val_loss: 0.2647 - val_mae: 0.4920
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7209 - mae: 0.6664 - val_loss: 0.2738 - val_mae: 0.5013
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5891 - mae: 0.6162 - val_loss: 0.2614 - val_mae: 0.4946
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5253 - mae: 0.5869 - val_loss: 0.2585 - val_mae: 0.4946
Epoch 9/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4441 - mae:

In [27]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2151 - mae: 0.4571 


In [28]:
print(test_acc)

0.4968787729740143


In [29]:
# pickle.dump(model, open("model.pkl", "wb"))

In [ ]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# Flatten the output
model.add(layers.Flatten())

# Summary of the CNN model
model.summary()
